In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from scipy.stats import spearmanr
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
df = pd.read_csv('ToN-IoT_balanced_smote.csv')

In [46]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, GRU
from keras.optimizers import Adam
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
X = data_balanced.drop('type', axis=1)
Y = data_balanced[['type']]
Y['type'] = Y['type'].astype('category')
# one hot encode the type
Y = pd.get_dummies(Y,columns=['type'])
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
# Y_train = np.asarray(Y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
# Y_test = np.asarray(Y_test).astype(np.float32)

# define the model2
model2 = Sequential()
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, clipvalue=0.5)
model2.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='softmax'))
model2.add(Dropout(0.2))
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(64))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train the model2
history = model2.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)

Epoch 1/10
75000/75000 [==============================] - 322s 4ms/step - loss: 0.3355 - accuracy: 0.8842 - val_loss: 0.2540 - val_accuracy: 0.9099
Epoch 2/10
75000/75000 [==============================] - 332s 4ms/step - loss: 0.2583 - accuracy: 0.9139 - val_loss: 0.2057 - val_accuracy: 0.9314
Epoch 3/10
75000/75000 [==============================] - 318s 4ms/step - loss: 0.2319 - accuracy: 0.9240 - val_loss: 0.1926 - val_accuracy: 0.9359
Epoch 4/10
75000/75000 [==============================] - 364s 5ms/step - loss: 0.2159 - accuracy: 0.9298 - val_loss: 0.1763 - val_accuracy: 0.9430
Epoch 5/10
75000/75000 [==============================] - 252s 3ms/step - loss: 0.2019 - accuracy: 0.9352 - val_loss: 0.1650 - val_accuracy: 0.9456
Epoch 6/10
75000/75000 [==============================] - 187s 2ms/step - loss: 0.1873 - accuracy: 0.9413 - val_loss: 0.1473 - val_accuracy: 0.9530
Epoch 7/10
75000/75000 [==============================] - 209s 3ms/step - loss: 0.1765 - accuracy: 0.9454 - val_

In [47]:
y_pred = model2.predict(X_test)
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))

18750/18750 [==============================] - 13s 649us/step
Classification Report
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     60144
           1       0.96      0.96      0.96     60079
           2       1.00      0.92      0.96     60047
           3       0.87      0.94      0.90     59974
           4       0.95      0.98      0.96     60175
           5       1.00      1.00      1.00     59762
           6       0.99      0.85      0.92     59631
           7       0.99      0.99      0.99     59909
           8       0.98      0.94      0.96     59992
           9       0.97      0.91      0.94     60287

   micro avg       0.97      0.95      0.96    600000
   macro avg       0.97      0.95      0.96    600000
weighted avg       0.97      0.95      0.96    600000
 samples avg       0.95      0.95      0.95    600000

Accuracy:
0.94872


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
